In [ ]:
# 用比較快快的json parser : ujson
try:
    import ujson as json
except ImportError:
    try:
        import simplejson as json
    except ImportError:
        import json

import math


# load tfidf
def readJson2Data(filename):
  f = open(filename, "r", encoding='UTF-8')
  docText = f.read()
  f.close()
  return json.loads(docText)


In [ ]:



DOC_NAME = 'data_user-docIDs.json'
name_docid = readJson2Data(DOC_NAME)
# map<userID,list[docID]>

NLVECTOR = 'name-docID_NLvector.json'
docID_name_NLvector = readJson2Data(NLVECTOR)
# 預先算好的tfidf 單位向量 字典
# map<docID,map{word:tf}>



In [ ]:
def findCommonWordSet(ida,idb):
    CommonWordSet = set()
    Aset = set( docID_name_NLvector[ida].keys() )
    Bset = set( docID_name_NLvector[idb].keys() )

    for token in Aset:
        if token in Bset:
            CommonWordSet.add(token)
    return CommonWordSet

In [ ]:
def cos(ida,idb):
    # get calNormalizedLength
    ida_NL = docID_name_NLvector[ida]
    idb_NL = docID_name_NLvector[idb]
    

    # findCommonWordSet
    commonWordSet = findCommonWordSet(ida,idb)

    # calCos  內積
    sum = 0
    for word in commonWordSet:
        sum = sum + ida_NL[word] * idb_NL[word]
    
    return sum

In [ ]:
import threading
import time
import queue

results = {}
docIDs = docID_name_NLvector.keys()


In [ ]:
#開啟多個執行緒，同時執行任務，有幾個執行緒就執行幾個任務


class MyThread(threading.Thread):
    def __init__(self, func):
        threading.Thread.__init__(self)
        self.func = func
    def run(self):
        self.func()

def worker():
    while not q.empty():
        item = q.get()  # 獲得任務
        print('Processing : ',item)
        # 拿到不同的docid  應該可以再改進
        doca , docb = item.split('+')
        try:
            results[docb] = cos(doca,docb)
        except:
            print('error ' + doca + '  '+ docb)

def main(name):
    threads = []
    targetList = getTargetDocs(name)
    # 產生task list
    for doc in targetList:
        for pttdoc in docIDs:
            q.put(doc + '+' +pttdoc)

            # 讓我知道現在queue到哪了
            if ( q.qsize() % 10000 ) == 0:
                print(q.qsize())

    for i in range(threadNum):   #開啟n個執行緒
        thread = MyThread(worker)
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()

if __name__ == '__main__':
    q = queue.Queue()
    threadNum = 24
    
    # name = ui()
    name = 'Kay731'
    if name == 0:
        print("retry")
    else:
        main(name)